In [22]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass # To get the password without showing the input
import seaborn as sns
import matplotlib.pyplot as plt

password = getpass.getpass()

········


# In order to optimize our inventory, we would like to know which films will be rented next month and we are asked to create a model to predict it.

# Instructions
1.Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features.


2.Read the data into a Pandas dataframe.


3.Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.


4.Create a query to get the list of films and a boolean indicating if it was rented last month. This would be our target variable.


5.Create a logistic regression model to predict this variable from the cleaned data.


6.Evaluate the results.

In [23]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = '''

select 
    f.film_id, 
    f.title,
    f.rental_duration,
    f.length,
    r.rental_id,
    r.rental_date,
    r.inventory_id,
    case when month(r.rental_date) = 2 AND YEAR(r.rental_date) = 2006 
            THEN 'True' 
            ELSE 'False' 
       END AS rented_feb_2006 
from sakila.inventory i
join sakila.film f on i.film_id = f.film_id
join sakila.rental r on i.inventory_id = r.inventory_id
;
'''

In [24]:
data = pd.read_sql_query(query,engine)
data.head()

,film_id,title,rental_duration,length,rental_id,rental_date,inventory_id,rented_feb_2006
0,1,ACADEMY DINOSAUR,6,86,4863,2005-07-08 19:03:15,1,False
1,1,ACADEMY DINOSAUR,6,86,11433,2005-08-02 20:13:10,1,False
2,1,ACADEMY DINOSAUR,6,86,14714,2005-08-21 21:27:43,1,False
3,1,ACADEMY DINOSAUR,6,86,972,2005-05-30 20:21:07,2,False
4,1,ACADEMY DINOSAUR,6,86,2117,2005-06-17 20:24:00,2,False


# 3.Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.

In [25]:
data.dtypes

film_id                     int64
title                      object
rental_duration             int64
length                      int64
rental_id                   int64
rental_date        datetime64[ns]
inventory_id                int64
rented_feb_2006            object
dtype: object

In [26]:
import datetime as dt
data['rental_date'] = pd.to_datetime(data['rental_date'])

data['rental_date'] = data['rental_date'].map(dt.datetime.toordinal)


In [27]:
data.dtypes

film_id             int64
title              object
rental_duration     int64
length              int64
rental_id           int64
rental_date         int64
inventory_id        int64
rented_feb_2006    object
dtype: object

In [28]:
data

,film_id,title,rental_duration,length,rental_id,rental_date,inventory_id,rented_feb_2006
0,1,ACADEMY DINOSAUR,6,86,4863,732135,1,False
1,1,ACADEMY DINOSAUR,6,86,11433,732160,1,False
2,1,ACADEMY DINOSAUR,6,86,14714,732179,1,False
3,1,ACADEMY DINOSAUR,6,86,972,732096,2,False
4,1,ACADEMY DINOSAUR,6,86,2117,732114,2,False
...,...,...,...,...,...,...,...,...
16039,1000,ZORRO ARK,3,50,711,732095,4581,False
16040,1000,ZORRO ARK,3,50,1493,732112,4581,False
16041,1000,ZORRO ARK,3,50,6712,732139,4581,False
16042,1000,ZORRO ARK,3,50,9701,732158,4581,False


In [29]:
data.isna().sum().sum()

0

In [30]:
import numpy as np
from sklearn.preprocessing import Normalizer

X = data.select_dtypes(include = np.number)        
X.head()

,film_id,rental_duration,length,rental_id,rental_date,inventory_id
0,1,6,86,4863,732135,1
1,1,6,86,11433,732160,1
2,1,6,86,14714,732179,1
3,1,6,86,972,732096,2
4,1,6,86,2117,732114,2


In [31]:
data.dtypes

film_id             int64
title              object
rental_duration     int64
length              int64
rental_id           int64
rental_date         int64
inventory_id        int64
rented_feb_2006    object
dtype: object

In [32]:
data["rented_feb_2006"] = data["rented_feb_2006"].replace({"True": True, "False": False}).astype(bool)

In [33]:
cat = data.select_dtypes(include = np.object)
cat

/var/folders/13/xty849950wj5f71_v7v6vysc0000gn/T/ipykernel_32452/2856053156.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cat = data.select_dtypes(include = np.object)


,title
0,ACADEMY DINOSAUR
1,ACADEMY DINOSAUR
2,ACADEMY DINOSAUR
3,ACADEMY DINOSAUR
4,ACADEMY DINOSAUR
...,...
16039,ZORRO ARK
16040,ZORRO ARK
16041,ZORRO ARK
16042,ZORRO ARK


In [34]:
cat = cat.drop(['rented_feb_2006'],axis=1)
cat                                     

KeyError: "['rented_feb_2006'] not found in axis"

In [35]:
categorical = pd.get_dummies(cat,columns = ['title'])  #get_dummies
categorical

,title_ACADEMY DINOSAUR,title_ACE GOLDFINGER,title_ADAPTATION HOLES,title_AFFAIR PREJUDICE,title_AFRICAN EGG,title_AGENT TRUMAN,title_AIRPLANE SIERRA,title_AIRPORT POLLOCK,title_ALABAMA DEVIL,title_ALADDIN CALENDAR,...,title_WORST BANGER,title_WRATH MILE,title_WRONG BEHAVIOR,title_WYOMING STORM,title_YENTL IDAHO,title_YOUNG LANGUAGE,title_YOUTH KICK,title_ZHIVAGO CORE,title_ZOOLANDER FICTION,title_ZORRO ARK
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16039,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
16040,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
16041,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
16042,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [36]:
X = np.concatenate((X,categorical),axis=1)
y = data['rented_feb_2006']

In [37]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.4)

In [38]:
from sklearn.linear_model import LogisticRegression

classification = LogisticRegression().fit(X_train,y_train)

In [39]:
predictions= classification.predict(X_test)
predictions

array([False, False, False, ..., False, False, False])

In [40]:
classification.score(X_test,y_test)

0.9889373636646931

In [41]:
y_test.value_counts()

False    6347
True       71
Name: rented_feb_2006, dtype: int64

In [42]:
pd.Series(predictions).value_counts()

False    6418
dtype: int64